In [30]:
import pandas as pd
from sqlalchemy import create_engine
import datetime

# STEP 1: EXTRACT

In [39]:
# DATASOURCE 1: Extract all records from winemag.csv - source kaggle.com

wine_file = "Resources/winemag.csv"
df_wine = pd.read_csv(wine_file)

print("Total records ", df_wine['winery'].count())
df_wine.head()
df_wine.count()

Total records  129971


Unnamed: 0               129971
country                  129908
description              129971
designation               92506
points                   129971
price                    120975
province                 129908
region_1                 108724
region_2                  50511
taster_name              103727
taster_twitter_handle     98758
title                    129971
variety                  129970
winery                   129971
dtype: int64

In [32]:
# Only keep required columns and rename them
wine_cols = ['Unnamed: 0','country','province','region_1','title','variety','winery','points','price']
df_wine_base = df_wine[wine_cols].copy()

df_wine_base = df_wine_base.rename(columns={'Unnamed: 0':'wine_id',
                                            'country': 'country_name',
                                            'province': 'province_name',
                                            'region_1':'region_name',
                                            'title':'wine_name',
                                            'variety':'wine_type',
                                            'winery':'winery_name',
                                            'points':'rating',
                                            'price':'price'})

df_wine_base = df_wine_base.replace(to_replace='US', value='United States')
df_wine_base.head()
df_wine_base.country_name.value_counts()

United States             54504
France                    22093
Italy                     19540
Spain                      6645
Portugal                   5691
Chile                      4472
Argentina                  3800
Austria                    3345
Australia                  2329
Germany                    2165
New Zealand                1419
South Africa               1401
Israel                      505
Greece                      466
Canada                      257
Hungary                     146
Bulgaria                    141
Romania                     120
Uruguay                     109
Turkey                       90
Slovenia                     87
Georgia                      86
England                      74
Croatia                      73
Mexico                       70
Moldova                      59
Brazil                       52
Lebanon                      35
Morocco                      28
Peru                         16
Ukraine                      14
Serbia  

In [34]:
# Drop any records that have emply values in any of their column fields
# df_wine_base = df_wine_base.dropna(how='any')
print("Total records ", df_wine_base['wine_id'].count())
df_wine_base.head()
df_wine_base.country_name.value_counts()

Total records  129971


United States             54504
France                    22093
Italy                     19540
Spain                      6645
Portugal                   5691
Chile                      4472
Argentina                  3800
Austria                    3345
Australia                  2329
Germany                    2165
New Zealand                1419
South Africa               1401
Israel                      505
Greece                      466
Canada                      257
Hungary                     146
Bulgaria                    141
Romania                     120
Uruguay                     109
Turkey                       90
Slovenia                     87
Georgia                      86
England                      74
Croatia                      73
Mexico                       70
Moldova                      59
Brazil                       52
Lebanon                      35
Morocco                      28
Peru                         16
Ukraine                      14
Serbia  

In [38]:
# Group df_wine_base and calculate average price and rating per unique wine in the database)
df_wine_data = df_wine_base.groupby([ #'wine_id' ,
                                     'country_name', 
                                     'province_name', 
                                     'region_name',
                                     'winery_name',
                                     'wine_type',
                                     'wine_name' 
                                     ], as_index=False)['rating','price'].mean().copy()

df_wine_data.reset_index(inplace=True)
print("Total records ", df_wine_data['country_name'].count())
# df_wine_data.head()
df_wine_data.country_name.value_counts()

Total records  99593


United States    49983
France           19899
Italy            17790
Spain             6019
Argentina         3494
Australia         2182
Canada             226
Name: country_name, dtype: int64

In [6]:
# DATASOURCE 2: extract country information from countries.csv - source: wikipedia
    
country_file = "Resources/countries.csv"
df_countries = pd.read_csv(country_file)
df_countries.head()
df_countries.reset_index(inplace=True)
df_countries = df_countries.rename(columns={'index':'country_id',
                                            'country':'country_abbr',
                                            'name': 'country_name'})
df_countries = df_countries.dropna(how='any')
print("Total records: ", df_countries['country_id'].count())
      
df_countries.head()

Total records:  243


,country_id,country_abbr,latitude,longitude,country_name
0,0,AD,42.546245,1.601554,Andorra
1,1,AE,23.424076,53.847818,United Arab Emirates
2,2,AF,33.939110,67.709953,Afghanistan
3,3,AG,17.060816,-61.796428,Antigua and Barbuda
4,4,AI,18.220554,-63.068615,Anguilla


In [7]:
# DATASOURCE 3: Directly load the temperatures by country table of the internet - source: Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_average_yearly_temperature'
tables = pd.read_html(url)

df_temps = pd.DataFrame(tables[0])

# Rename columns
df_temps = df_temps.rename(columns={'Country':'country_name','Average yearly temperature (1961–1990, degrees Celsius)':'avg_temp'})

# # Reset index for fast lookup by df_countries
df_temps = df_temps.set_index('country_name')

print("Total records: ", df_temps['avg_temp'].count())
df_temps.head()

Total records:  191


,avg_temp
country_name,
Burkina Faso,28.25
Mali,28.25
Kiribati,28.20
Djibouti,28.00
Tuvalu,28.00


# STEP 2: TRANSFORM

In [8]:
# Add temperature to df_countries when found
df_countries['temperature'] = ""
list_temp = []

for index, row in df_countries.iterrows():
    try:
        list_temp = df_temps.loc[str(row[4]),'avg_temp']
        df_countries.iloc[index, 5] = list_temp
    except:
        pass

# Two dataframes - one for saving to the database later that includes the index_id and the other for lookup for df_wine_data
df_countries.head()
df_countries_lookup = df_countries.copy()
df_countries_lookup = df_countries_lookup.set_index("country_name")
df_countries_lookup.head()

,country_id,country_abbr,latitude,longitude,temperature
country_name,,,,,
Andorra,0,AD,42.546245,1.601554,07.60
United Arab Emirates,1,AE,23.424076,53.847818,27.00
Afghanistan,2,AF,33.939110,67.709953,12.60
Antigua and Barbuda,3,AG,17.060816,-61.796428,26.00
Anguilla,4,AI,18.220554,-63.068615,


In [9]:
# from df_wine_base, create seperate tables, to normalize the database
# df_wineries, df_provinces and df_regions. df_countries was created by pulling so called 'master data' of the internet.

# For the wineries

df_winery_base = df_wine_base.groupby(['winery_name']).count()
df_winery_base.reset_index(inplace=True)
df_wineries = df_winery_base[['winery_name']]
df_wineries.reset_index(inplace=True)
df_wineries = df_wineries.rename(columns={'index':'winery_id'})
print("Total records: ", df_wineries['winery_id'].count())

# Two dataframes - one for saving to the database later that includes the index_id and the other for lookup for df_wine_data
df_wineries.head()
df_wineries_lookup = df_wineries.copy()
df_wineries_lookup = df_wineries_lookup.set_index("winery_name")
df_wineries_lookup.head()

Total records:  16757


,winery_id
winery_name,
1+1=3,0
10 Knots,1
100 Percent Wine,2
1000 Stories,3
1070 Green,4


In [10]:
# For the provinces

df_provinces_base = df_wine_base.groupby(['province_name']).count()
df_provinces_base.reset_index(inplace=True)
df_provinces = df_provinces_base[['province_name']]
df_provinces.reset_index(inplace=True)
df_provinces = df_provinces.rename(columns={'index':'province_id'})
print("Total records: ", df_provinces['province_id'].count())

# Two dataframes - one for saving to the database later that includes the index_id and the other for lookup for df_wine_data
df_provinces.head()
df_provinces_lookup = df_provinces.copy()
df_provinces_lookup = df_provinces_lookup.set_index("province_name")
df_provinces_lookup.head()

Total records:  425


,province_id
province_name,
Achaia,0
Aconcagua Costa,1
Aconcagua Valley,2
Aegean,3
Agioritikos,4


In [11]:
# For the regions

df_regions_base = df_wine_base.groupby(['region_name']).count()
df_regions_base.reset_index(inplace=True)
df_regions = df_regions_base[['region_name']]
df_regions.reset_index(inplace=True)
df_regions = df_regions.rename(columns={'index':'region_id'})
print("Total records: ", df_regions['region_id'].count())

# Two dataframes - one for saving to the database later that includes the index_id and the other for lookup for df_wine_data
df_regions.head()
df_regions_lookup = df_regions.copy()
df_regions_lookup = df_regions_lookup.set_index("region_name")
df_regions_lookup.head()

Total records:  1229


,region_id
region_name,
Abruzzo,0
Adelaida District,1
Adelaide,2
Adelaide Hills,3
Adelaide Plains,4


In [12]:
# For the wine_types

df_wine_types_base = df_wine_base.groupby(['wine_type']).count()
df_wine_types_base.reset_index(inplace=True)
df_wine_types = df_wine_types_base[['wine_type']]
df_wine_types.reset_index(inplace=True)
df_wine_types = df_wine_types.rename(columns={'index':'wine_type_id'})
print("Total records: ", df_wine_types['wine_type_id'].count())

# Two dataframes - one for saving to the database later that includes the index_id and the other for lookup for df_wine_data
df_wine_types.head()
# df_wine_types_lookup = df_wine_types.copy()
# df_wine_types_lookup = df_wine_types_lookup.set_index("wine_type")
# df_wine_types_lookup.head()

Total records:  707


,wine_type_id,wine_type
0,0,Abouriou
1,1,Agiorgitiko
2,2,Aglianico
3,3,Aidani
4,4,Airen


In [13]:
# For reference for the part below: df_wine_base before
df_wine_data.head()

,index,country_name,province_name,region_name,winery_name,wine_type,wine_name,rating,price
0,0,Argentina,Mendoza Province,Agrelo,Cadus,Cabernet Sauvignon,Cadus 2014 Cabernet Sauvignon (Agrelo),88.0,20.0
1,1,Argentina,Mendoza Province,Agrelo,Casarena,Cabernet Sauvignon,Casarena 2009 Estate Bottled Cabernet Sauvigno...,84.0,17.0
2,2,Argentina,Mendoza Province,Agrelo,Casarena,Cabernet Sauvignon,Casarena 2012 Owen's Vineyard Cabernet Sauvign...,87.0,40.0
3,3,Argentina,Mendoza Province,Agrelo,Casarena,Malbec,Casarena 2011 Lauren's Vineyard Malbec (Agrelo),89.0,40.0
4,4,Argentina,Mendoza Province,Agrelo,El Enemigo,Cabernet Franc,El Enemigo 2013 Gran Enemigo Agrelo Single Vin...,93.0,120.0


In [14]:
# Now replace all _name strings in df_wine_base with their correlating _id keys
print("Start time: ", datetime.datetime.now().time())
# limit = 3

df_wine_data = df_wine_data.merge(df_countries, on='country_name', how='inner')
df_wine_data = df_wine_data.merge(df_provinces, on='province_name', how='inner')
df_wine_data = df_wine_data.merge(df_regions, on='region_name', how='inner')
df_wine_data = df_wine_data.merge(df_wineries, on='winery_name', how='inner')
df_wine_data = df_wine_data.merge(df_wine_types, on='wine_type', how='inner')
    
print("Finish time: ", datetime.datetime.now().time())
df_wine_data.head()

Start time:  20:55:12.535766
Finish time:  20:55:12.838986


,index,country_name,province_name,region_name,winery_name,wine_type,wine_name,rating,price,country_id,country_abbr,latitude,longitude,temperature,province_id,region_id,winery_id,wine_type_id
0,0,Argentina,Mendoza Province,Agrelo,Cadus,Cabernet Sauvignon,Cadus 2014 Cabernet Sauvignon (Agrelo),88.0,20.0,10,AR,-38.416097,-63.616672,14.80,216,9,1935,80
1,1,Argentina,Mendoza Province,Agrelo,Casarena,Cabernet Sauvignon,Casarena 2009 Estate Bottled Cabernet Sauvigno...,84.0,17.0,10,AR,-38.416097,-63.616672,14.80,216,9,2406,80
2,2,Argentina,Mendoza Province,Agrelo,Casarena,Cabernet Sauvignon,Casarena 2012 Owen's Vineyard Cabernet Sauvign...,87.0,40.0,10,AR,-38.416097,-63.616672,14.80,216,9,2406,80
3,122,Argentina,Mendoza Province,Luján de Cuyo,Casarena,Cabernet Sauvignon,Casarena 2013 Reservado Cabernet Sauvignon (Lu...,86.0,20.0,10,AR,-38.416097,-63.616672,14.80,216,589,2406,80
4,123,Argentina,Mendoza Province,Luján de Cuyo,Casarena,Cabernet Sauvignon,Casarena 2015 Estate Cabernet Sauvignon (Luján...,86.0,20.0,10,AR,-38.416097,-63.616672,14.80,216,589,2406,80


In [15]:
wine_cols = ['index','wine_name','country_id','province_id','region_id','winery_id','wine_type_id','rating','price']
df_wine_final = df_wine_data[wine_cols].copy()

df_wine_final = df_wine_final.rename(columns={'index':'wine_id'})

In [16]:
df_wine_final.head()

,wine_id,wine_name,country_id,province_id,region_id,winery_id,wine_type_id,rating,price
0,0,Cadus 2014 Cabernet Sauvignon (Agrelo),10,216,9,1935,80,88.0,20.0
1,1,Casarena 2009 Estate Bottled Cabernet Sauvigno...,10,216,9,2406,80,84.0,17.0
2,2,Casarena 2012 Owen's Vineyard Cabernet Sauvign...,10,216,9,2406,80,87.0,40.0
3,122,Casarena 2013 Reservado Cabernet Sauvignon (Lu...,10,216,589,2406,80,86.0,20.0
4,123,Casarena 2015 Estate Cabernet Sauvignon (Luján...,10,216,589,2406,80,86.0,20.0


# STEP 3: LOAD

In [17]:
# Create connection
connection_string = "postgres:postgres@localhost:5432/wine_db"
engine = create_engine(f'postgresql://{connection_string}')

In [18]:
# Load all 5 dataframes into their respective tables
df_countries.to_sql(name='countries', con=engine, if_exists='replace', index=False)
df_provinces.to_sql(name='provinces', con=engine, if_exists='replace', index=False)
df_regions.to_sql(name='regions', con=engine, if_exists='replace', index=False)
df_wineries.to_sql(name='wineries', con=engine, if_exists='replace', index=False)
df_wine_types.to_sql(name='wine_types', con=engine, if_exists='replace', index=False)
df_wine_final.to_sql(name='wines', con=engine, if_exists='replace', index=False)

In [19]:
# Create one sql statement, that shows all data is now connected

In [20]:
sql_statement = 

SyntaxError: invalid syntax (<ipython-input-20-9c1eab9fcdcb>, line 1)